<h1> Capstone Project by Jason </h1>


In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


<h3>Import library</h3>

In [5]:
import requests
from bs4 import BeautifulSoup


<h3>Retrieve data from wikipedia</h3>

In [6]:




website_url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text


soup = BeautifulSoup(website_url,"lxml")
#print(soup.prettify())




table = soup.find("table",{"class":"wikitable sortable"})


parsed_table_data = []



rows = table.findAll('tr')
for row in rows:
                children = row.findChildren(recursive=False)
                row_text = []
            
                for child in children:
                    clean_text = child.text.strip()
                    row_text.append(clean_text)
  
            
                parsed_table_data.append(row_text)
                
                


df=pd.DataFrame(parsed_table_data,columns=["Postcode","Borough","Neighbour"])


df.head()


,Postcode,Borough,Neighbour
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


<h3>Data cleansing</h3>

In [7]:
df=df[df.Borough!="Not assigned"]
df=df[df.Postcode!="Postcode"]
df.shape
df.loc[df.Neighbour=="Not assigned","Neighbour"] = df.Borough
df.head()

,Postcode,Borough,Neighbour
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


<h3>Group data by postcode</h3>

In [8]:
df1 = (df.groupby(['Postcode','Borough'])['Neighbour']
       .apply(lambda x: ','.join(set(x.dropna())))
       .reset_index())
df1.columns=["PostCode","Borough","Neighbour"]
df1.head()

,PostCode,Borough,Neighbour
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Morningside,Guildwood,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [9]:
df=df1
df.shape

(103, 3)

<h3>Get GEO Data</h3>

In [10]:
geo_df = pd.read_csv("http://cocl.us/Geospatial_data")
geo_df.columns=["PostCode","Latitude","Longitude"]
geo_df.head()



,PostCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
df=pd.merge(df,geo_df,on="PostCode")
df

,PostCode,Borough,Neighbour,Latitude_x,Longitude_x,Latitude_y,Longitude_y
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,43.784535,-79.160497
2,M1E,Scarborough,"Morningside,Guildwood,West Hill",43.763573,-79.188711,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,43.744734,-79.239476
6,M1K,Scarborough,"Ionview,Kennedy Park,East Birchmount Park",43.727929,-79.262029,43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile,Clairlea,Oakridge",43.711112,-79.284577,43.711112,-79.284577
8,M1M,Scarborough,"Cliffside,Scarborough Village West,Cliffcrest",43.716316,-79.239476,43.716316,-79.239476
9,M1N,Scarborough,"Cliffside West,Birch Cliff",43.692657,-79.264848,43.692657,-79.264848
